In [3]:
import hashlib, pandas as pd, numpy as np
from pathlib import Path

def df_md5(df: pd.DataFrame) -> str:
    b = df.to_csv(index=False).encode("utf-8")
    return hashlib.md5(b).hexdigest()

def assert_monotonic_ts(df, col="timestamp"):
    ts = pd.to_datetime(df[col], utc=True, errors="coerce")
    assert ts.notna().all(), f"{col} has NaT"
    assert (ts.sort_values().values==ts.values).all(), f"{col} not monotonic ↑"

def assert_zs(df):
    if not {"zs_low","zs_high"}.issubset(df.columns): return
    low, high = df["zs_low"].astype(float), df["zs_high"].astype(float)
    mid = 0.5*(low+high)
    assert (low <= mid).all() and (mid <= high).all(), "zs low/mid/high invariant broken"

def jaccard_events(a: pd.DataFrame, b: pd.DataFrame, keys=("ts","etype","side","tag")):
    A = set(map(tuple, a[list(keys)].astype(str).values))
    B = set(map(tuple, b[list(keys)].astype(str).values))
    return len(A & B)/max(1,len(A | B))

pA = Path("/Users/cc/PycharmProjects/ChanLab/chan_data/signals/mmd/BTCUSDT/15m_mmd123.csv")
pB = Path("/Users/cc/PycharmProjects/ChanLab/chan_data/signals/mmd/BTCUSDT/15m_mmd123.csv")
a = pd.read_csv(pA, parse_dates=["ts"])
b = pd.read_csv(pB, parse_dates=["ts"])

assert len(a)==len(b), "row count mismatch"
assert df_md5(a)==df_md5(b), "content hash mismatch"
assert_monotonic_ts(a, "ts"); assert_monotonic_ts(b, "ts")
print("OK: deterministic & monotonic")

keys = ["ts","side","tag"] if "etype" not in a.columns else ["ts","etype","side","tag"]
print("Jaccard =", jaccard_events(a,b, keys=keys))

OK: deterministic & monotonic
Jaccard = 1.0
